# TODO
- vP is not allow to be 0 or 1
- add everything to dictionaries
- attacker only works with dict
- optimise
- try catch when non invertible prime
- t must not be prime --> optimise
- when there is no R large/ small en
- when delta of bounds is small then the possibility of a new generation of p is high 
- when p is smaller than there is is higher chance of success
- Problem when m>=n 
- Use secret module instead of the random module for cryptographic purposes
- camelCase to snake_case if required
- gen_prime() is not used

### Import and Configuration

In [236]:
import sympy
import random
import coloredlogs, logging

In [237]:
coloredlogs.install(fmt='%(levelname)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [238]:
MIN_PRIME = 0
MAX_PRIME = 1024
OPT_RANGE_REDUCTION = 24

MAX_ITERS = 10

msg = 12345678900987654321

In [239]:
def gen_prime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def gen_prime_with_optimized_bit_limit():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def gen_prime_with_boundaries(lb, ub):
    return sympy.randprime(lb, ub)

def gen_prime_half_size():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def gen_random_number_with_bit_limit():
    return random.randint(0, pow(2, MAX_PRIME))

def fermat_prime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def modular_multiplicative_inverse(E, phiN):
    return pow(E, -1, phiN)

def calc_n(P, Q):
    return P*Q

def calc_phi_n(P, Q):
    return (P-1) * (Q-1)

def to_binary(dec):
    return "{0:b}".format(dec)

def to_decimal(bin):
    return int(bin, 2)

def concatenate_in_binary(prev, tail):
    prev = to_binary(prev)
    tail = to_binary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return to_decimal(prev + tail)

def split_in_binary(bin):
    bin = to_binary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(to_decimal(prev), to_decimal(tail))



# Voraussetzungen

In [240]:
def create_attacker_key():
    a_P = gen_prime_half_size()
    a_Q= gen_prime_half_size()


    a_PhiN = calc_phi_n(a_P, a_Q)
    a_N = calc_n(a_P, a_Q)

    a_E = fermat_prime()
    a_D = modular_multiplicative_inverse(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

Attacker = {}

created = False
while not created:
    try:
        Attacker = create_attacker_key()
        created = True
    except ValueError:
            print("Trying to generate Key failed. Trying again ...")

print(Attacker)

{'P': 13273137527310972810899646897937614764526024704880566035087586147893132410277970055919221675684709602911504887579936240875007061278601018367739924881148547, 'Q': 1396982511312765453429241450073025985395792304270226859990296485380847133957351696568627443054342715115085992390740768624076657306263011722797554469862553, 'PhiN': 18542340995902592752381413974807621712158636681251497893381719843179638708135497305258017282057617158487018552516832330509370460999193664166125757573686313051309184548155673965210339128910202287981512955981524618272324938631174010684147713120686820761274620462380064070429015600440528535071336401236514649392, 'N': 18542340995902592752381413974807621712158636681251497893381719843179638708135497305258017282057617158487018552516832330509370460999193664166125757573686327721429223171893938294098687139550952209798522106774419696154958212610718246005900200969805559813592647053260034747438514684159113399101426938715865660491, 'E': 5, 'D': 741693639836103710095256558992

# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [241]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
User = {}

valid = False
while not valid:
    # Schritt 1
    P = gen_prime_with_optimized_bit_limit()
    vP = pow(P, Attacker["E"] ,Attacker["N"])

    # Schritt 2
    t = gen_random_number_with_bit_limit() 
    tempN = concatenate_in_binary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    ub = int(int(tempN)//int(P)) 
    lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    logger.info("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    R = 0
    Q = 0
    try:
        for i in range(MAX_ITERS):
            Q = int(gen_prime_with_boundaries(lb, ub))
            R = tempN - P*Q

            if (R < pow(2,MAX_PRIME+1) and R >= 0):
                valid = True
                break
            r_too_large += 1
    except ValueError:
        pass
    User["P"] = P
    User["Q"] = Q

Attacker["vP"] = vP
Attacker["R"] = R

logger.info("Amount of failed attempts: {0}".format(r_too_large))

logger.info("tempN = P * Q + R")
logger.info("{0} = {1} * {2} + {3}".format(tempN, P, Q, R))

print(User["P"])
x = pow(User["P"], Attacker["E"], Attacker["N"])
# print(x)

y = pow(x, Attacker["D"], Attacker["N"])

print(y)



INFO Bounds: [897660470760517902474846772983569085015752983034622688817779341266642002898723199261430009297735545407889062239389100712677350570675432316305270629869330984893860511924502072559335763663211304815930449169777331740752220824325334974322433117935185203296159827072692197079363148821491271569886255321212322468670536616, 897660470760517902474846772983569085015752983034622688817779341266642002898723199261430009297735545407889062239389100712677350570675432316305270629869330984893860511924502072559335763663211304815930449169777331740752220824325334974322433117935185203296159827072692197079363148821491271569886255321212322468732650275] 	 P: 2894199352290041620657544994574413974644827484725270693447665013213813690566851078991452091462045068114612767445673417798352440478374244997201227722117223295000668606757124042136777392296169575748327174761712939844163219952564770176984370917360671073071737337414413997718956982475330793418671641946441 	 deltaB: 62113659
INFO Amount of failed atte

2894199352290041620657544994574413974644827484725270693447665013213813690566851078991452091462045068114612767445673417798352440478374244997201227722117223295000668606757124042136777392296169575748327174761712939844163219952564770176984370917360671073071737337414413997718956982475330793418671641946441
2894199352290041620657544994574413974644827484725270693447665013213813690566851078991452091462045068114612767445673417798352440478374244997201227722117223295000668606757124042136777392296169575748327174761712939844163219952564770176984370917360671073071737337414413997718956982475330793418671641946441


## Schritt 4

In [242]:
User["N"] = calc_n(User["P"], User["Q"])

## Schritt 5

In [243]:
def create_user_key():
    PhiN = calc_phi_n(P, Q)
    E = fermat_prime()
    D = modular_multiplicative_inverse(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        User.update(create_user_key())
        created = True
    except ValueError:
            logger.info("Trying to generate Key failed. Trying again ...")
logger.info("User Key successfully generated!")

INFO User Key successfully generated!


## Schritt 6

In [244]:
cipher = pow(msg, User["E"], User["N"])
signature = pow(msg, User["D"], User["N"])

# Angriff

## Schritt 1

In [245]:
pub_User = {
    "N" : User["N"],
    "E" : User["E"]
}

collection_User = {
    "N" : pub_User["N"],
    "E" : pub_User["E"]
}

## Schritt 2

In [246]:
collection_User["vP"] = split_in_binary(User["N"])[0]
collection_User["vP'"] = collection_User["vP"]+1

### Validation test for vP

In [247]:
testVar = "vP"
altTestVar = "vP'"

if(collection_User[testVar] == Attacker[testVar] or collection_User[altTestVar] == Attacker[testVar]):
    logger.info("Success!!!")
    logger.info("The variable {0} or {1} is in the expected state: {2}".format(testVar, altTestVar, Attacker[testVar]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(Attacker[testVar], collection_User[testVar]))

INFO Success!!!
INFO The variable vP or vP' is in the expected state: 14451901176395405480437988696773040252183252594477077591762761124787282849840949498896020684017049255571685446274746734817188537498692986842811119346621978095329570261079032619184698225395131641621376347279854728923958228155560158077114326917561669165023107045177600519851378414690769344913970151764162690864
INFO Current vP: 14451901176395405480437988696773040252183252594477077591762761124787282849840949498896020684017049255571685446274746734817188537498692986842811119346621978095329570261079032619184698225395131641621376347279854728923958228155560158077114326917561669165023107045177600519851378414690769344913970151764162690864 	 Current vP': 1445190117639540548043798869677304025218325259447707759176276112478728284984094949889602068401704925557168544627474673481718853749869298684281111934662197809532957026107903261918469822539513164162137634727985472892395822815556015807711432691756166916502310704517760051985137841469

## Schritt 3

In [248]:
collection_User["P"] = pow(collection_User["vP"], Attacker["D"], Attacker["N"])
collection_User["P'"] = pow(collection_User["vP'"], Attacker["D"], Attacker["N"])

print(Attacker)
print(User)
print(collection_User)


{'P': 13273137527310972810899646897937614764526024704880566035087586147893132410277970055919221675684709602911504887579936240875007061278601018367739924881148547, 'Q': 1396982511312765453429241450073025985395792304270226859990296485380847133957351696568627443054342715115085992390740768624076657306263011722797554469862553, 'PhiN': 18542340995902592752381413974807621712158636681251497893381719843179638708135497305258017282057617158487018552516832330509370460999193664166125757573686313051309184548155673965210339128910202287981512955981524618272324938631174010684147713120686820761274620462380064070429015600440528535071336401236514649392, 'N': 18542340995902592752381413974807621712158636681251497893381719843179638708135497305258017282057617158487018552516832330509370460999193664166125757573686327721429223171893938294098687139550952209798522106774419696154958212610718246005900200969805559813592647053260034747438514684159113399101426938715865660491, 'E': 5, 'D': 741693639836103710095256558992

### Validation test for P and P'

In [249]:
testVar = "P"
altTestVar = "P'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state: {2}".format(testVar, altTestVar, User["P"]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} or {1} doesn't have the expected value.".format(testVar, altTestVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(User[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables P or P' are in the expected state: 2894199352290041620657544994574413974644827484725270693447665013213813690566851078991452091462045068114612767445673417798352440478374244997201227722117223295000668606757124042136777392296169575748327174761712939844163219952564770176984370917360671073071737337414413997718956982475330793418671641946441
INFO Current P: 2894199352290041620657544994574413974644827484725270693447665013213813690566851078991452091462045068114612767445673417798352440478374244997201227722117223295000668606757124042136777392296169575748327174761712939844163219952564770176984370917360671073071737337414413997718956982475330793418671641946441 	 Current P': 11319854189595298805444817639423267228584662124851320126676781390416911557405526901692871347548986759548409760414379649087469997135527653634729937814275775832694698362776526313179302721906735677700589504689074305409443214963529732023233159495099855877638037083997259358778521364898057361793278959

# Schritt 4

In [250]:
# Potential to optimize when N/P not hole Number
collection_User["Q"] = int(pub_User["N"]//collection_User["P"])
collection_User["Q'"] = int(pub_User["N"]//collection_User["P'"])

### Validation test for Q and Q'

In [251]:
testVar = "Q"
altTestVar = "Q'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables Q or Q' are in the expected state!
INFO Current Q: 897660470760517902474846772983569085015752983034622688817779341266642002898723199261430009297735545407889062239389100712677350570675432316305270629869330984893860511924502072559335763663211304815930449169777331740752220824325334974322433117935185203296159827072692197079363148821491271569886255321212322468700383701 	 Current Q': 229508994509791236322096779931479046187024006871159328374073049757136006053442912839206931560926894361967581453448039110158380657920136868398240865602736253637173715953892970519437278959670182996271320367597212495056603800093012335298300194453979646544955538229631795394800760688920534779987807417939895956673


## Schritt 5

In [252]:
collection_User["PhiN"] = int(calc_phi_n(collection_User["P"], collection_User["Q"]))
collection_User["PhiN'"] = int(calc_phi_n(collection_User["P'"], collection_User["Q'"]))

collection_User["D"] = modular_multiplicative_inverse(pub_User["E"], collection_User["PhiN"])
collection_User["D'"] = modular_multiplicative_inverse(pub_User["E"], collection_User["PhiN'"])

### Validation test for PhiN and PhiN'

In [253]:
testVar = "PhiN"
altTestVar = "PhiN'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables PhiN or PhiN' are in the expected state!
INFO Current PhiN: 2598008353051464758266696120517033958298563544739687323974796696868670375616114468308412588819182305745784776994430207535117308155017821389164960408026098664599898668175172355730131643290686837043696307933999134047006386337749658640614488131162453304759343312844073640144783538965950803125071099469444556706604869980478250783615936903445843700404648742014521287468534280140777055181503046335593498995202857080899078571081644761735052559588846958528110343290660211180549233335254186867292992904002787120377661944940221941930335275003169043004248655943874718422124440149306215443712950893157107125425358285442849028000 	 Current PhiN': 2598008353051464758266696120517033958298563544739687323974796696868670375616114468308412588819182305745784776994430207535117308155017821389164960408026098664599898668175172355730131643290686837043696307933999134047006386337749658640614488131162453304759343312844073640144

### Validation test for D and D'

In [254]:
testVar = "D"
altTestVar = "D'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables D or D' are in the expected state!
INFO Current D: 780468871856771566607484523562863185238599860671604803918638286280274995119246557701085296072021319796801969718881577215182104200009015785127176106828475983164362192029430601638995251432122960277649158652434120437576662560349348906974961355056325139449501062655814605904611109986139728884849776559110493804105111313847991232858241386621935254490848290778978212424744845009866467223300303282957181564880202789397455771662655020355520008440807771480254458377809759948161394416962699407102315398843506306757944495048340473209399894628414363163825740241741389692765704528122141227944833867654563454008795854613406345473 	 Current D': 13404893794106425500219468287422906648132494051508693846255969956606815806561012976768691226470642487265775399250294866685992916255234086130676279994110582776667466235003654913867647514819807954992231968626604928177902704428186323287971514740567672993948028460842325730731625703058368

## Schritt 6

In [255]:
collection_User["Sig"] = pow(msg, collection_User["D"], User["N"])
collection_User["Sig'"] = pow(msg, collection_User["D'"], User["N"])

### Validation test for signature and signature'

In [256]:
testVar = "Sig"
altTestVar = "Sig'"

if(collection_User[testVar] == signature):
    collection_User["correct_key"] = "D"
    logger.info("D is the right privat Key: {0}".format(collection_User[testVar]))
elif(collection_User[altTestVar] == signature):
    collection_User["correct_key"] = "D'"
    logger.info("D' is the right privat Key: {0}".format(collection_User[altTestVar]))
else:
    logger.error("Message was incorrectly signed!")

INFO D is the right privat Key: 857581573749794746944772214452624799210154228781772738282763601317466934775498787377319081079743115277403266819665333173838770768447937217169777338734262274935536635649251961098234102191706044878589151081580302576140197712307145125628114386685157500873490435629068452767484003313302547254290082281633628369939814500063156240076162299658330621796491358853383668594457355199039832157987139700618305823669374199219187388757944139428302812431688504708474944679382901492226617798661272726768308756134997843921251665283868543126771739239308162588309768301735216592520363948144377573792868731920107480389215631386483459053


# Result

In [257]:
def sort_dic(dict):
    return {key: dict[key] for key in sorted(dict)}

In [258]:
print(sort_dic(Attacker))
print(sort_dic(User))
print(sort_dic(collection_User))



{'D': 7416936398361037100952565589923048684863454672500599157352687937271855483254198922103206912823046863394807421006732932203748184399677465666450303029474525220523673819262269586084135651564080915192605182392609847308929975452469604273659085248274728304509848184952025628171606240176211414028534560494605859757, 'E': 5, 'N': 18542340995902592752381413974807621712158636681251497893381719843179638708135497305258017282057617158487018552516832330509370460999193664166125757573686327721429223171893938294098687139550952209798522106774419696154958212610718246005900200969805559813592647053260034747438514684159113399101426938715865660491, 'P': 13273137527310972810899646897937614764526024704880566035087586147893132410277970055919221675684709602911504887579936240875007061278601018367739924881148547, 'PhiN': 185423409959025927523814139748076217121586366812514978933817198431796387081354973052580172820576171584870185525168323305093704609991936641661257575736863130513091845481556739652103391289102022